In [70]:
import pandas as pd
import urllib
import os
import httplib2
import numpy as np
from apiclient import discovery
from oauth2client import client
from oauth2client import tools
from oauth2client.file import Storage

In [2]:
def download(url,filename):
    #filename = url.split("/")[-1]
    if not os.path.exists(filename):
        urllib.urlretrieve(url, filename)

In [267]:
url =  'http://image-net.org/archive/words.txt'
filename = 'all_imagenet_synsets_and_descriptions.txt'

download(url,filename) 

df = pd.read_csv(filename, sep=r'\t',engine='python',
                          names=['wnid','description'],na_filter=False)
#print 'Original wnids and dscriptions form Image-Net'
#df.head()

filename = 'all_wnids.csv'
sep= ';'
if not os.path.exists(filename):
    df_temp = pd.concat([pd.Series(row['wnid'], row['description'].split(','))              
                        for _, row in df.iterrows()]).reset_index()
    all_wnids = df_temp.rename(columns={'index':'description',0:'wnid'})
    all_wnids['description'] = all_wnids['description'].str.lower().str.strip()
    all_wnids.to_csv('all_wnids.csv',sep=sep,index=False)
else: 
    all_wnids = pd.read_csv(filename,sep=sep)
print 'All wnids descriptions split in one column'
all_wnids.head()

All wnids descriptions split in one column


,description,wnid
0,entity,n00001740
1,physical entity,n00001930
2,abstraction,n00002137
3,abstract entity,n00002137
4,thing,n00002452


In [311]:
veg_wnid = 'n07707451'
fruit_wnid = 'n07705931'
ingred_wnid = 'n07809096'

print 'Vegetables Master Synset:','\t',veg_wnid
print 'Fruits Master Synset:','\t\t',fruit_wnid
print 'Flavorer Master Synset:','\t',flavorer_wnid

url1 = 'http://www.image-net.org/api/text/wordnet.structure.hyponym?wnid='
param1 = '&full=1'

response = urllib.urlopen(url1+veg_wnid+param1)
veg_wnids = pd.DataFrame(response.read().replace('-','').splitlines())
veg_wnids.columns = ['wnid']
veg_wnids['group']='vegetables'

response = urllib.urlopen(url1+fruit_wnid+param1)
fruit_wnids = pd.DataFrame(response.read().replace('-','').splitlines())
fruit_wnids.columns = ['wnid']
fruit_wnids['group']='fruits'

response = urllib.urlopen(url1+ingred_wnid+param1)
ingred_wnids = pd.DataFrame(response.read().replace('-','').splitlines())
ingred_wnids.columns = ['wnid']
ingred_wnids['group']='flavorers'


all_fr_veg = veg_wnids.append(fruit_wnids).append(ingred_wnids)
all_fr_veg_wnids = pd.merge(all_wnids,all_fr_veg)
print 'number of all edible fruits, vegetables and flavorers from imagenet',all_fr_veg_wnids.shape
all_fr_veg_wnids.head(10)

Vegetables Master Synset: 	n07707451
Fruits Master Synset: 		n07705931
Flavorer Master Synset: 	n07809368
number of all edible fruits, vegetables and flavorers from imagenet (974, 3)


,description,wnid,group
0,powdered mustard,n03994417,flavorers
1,dry mustard,n03994417,flavorers
2,relish,n07582441,flavorers
3,dip,n07582609,flavorers
4,bean dip,n07582811,flavorers
5,cheese dip,n07582892,flavorers
6,clam dip,n07582970,flavorers
7,guacamole,n07583066,flavorers
8,caramel,n07595180,flavorers
9,caramelized sugar,n07595180,flavorers


## Read the ingredients from google doc and match with imagenet "wnid"

In [270]:
# If modifying these scopes, delete your previously saved credentials
# at ~/.credentials/sheets.googleapis.com-python-quickstart.json
SCOPES = 'https://www.googleapis.com/auth/spreadsheets.readonly'
CLIENT_SECRET_FILE = '../client_secret.json'
APPLICATION_NAME = 'Test'


def get_credentials():
    """Gets valid user credentials from storage.

    If nothing has been stored, or if the stored credentials are invalid,
    the OAuth2 flow is completed to obtain the new credentials.

    Returns:
        Credentials, the obtained credential.
    """
    home_dir = os.path.expanduser('~')
    credential_dir = os.path.join(home_dir, '.credentials')
    if not os.path.exists(credential_dir):
        os.makedirs(credential_dir)
    credential_path = os.path.join(credential_dir,
                                   'sheets.googleapis.com-python-quickstart.json')

    store = Storage(credential_path)
    credentials = store.get()
    if not credentials or credentials.invalid:
        flow = client.flow_from_clientsecrets(CLIENT_SECRET_FILE, SCOPES)
        flow.user_agent = APPLICATION_NAME
        if flags:
            credentials = tools.run_flow(flow, store, flags)
        else: # Needed only for compatibility with Python 2.6
            credentials = tools.run(flow, store)
        print 'Storing credentials to ' + credential_path 
    return credentials


"""Shows basic usage of the Sheets API.

ingrdients spreadsheet:
https://docs.google.com/spreadsheets/d/1LHojuRe1U7nhJUG-m7aMfA4BNQX-BmbmGlCSErE1NIg/edit
"""
credentials = get_credentials()
http = credentials.authorize(httplib2.Http())
discoveryUrl = ('https://sheets.googleapis.com/$discovery/rest?'
                'version=v4')
service = discovery.build('sheets', 'v4', http=http,
                          discoveryServiceUrl=discoveryUrl)

spreadsheetId = '1LHojuRe1U7nhJUG-m7aMfA4BNQX-BmbmGlCSErE1NIg'
rangeName = 'All_in_one'
result = service.spreadsheets().values().get(
    spreadsheetId=spreadsheetId, range=rangeName).execute()
values = result.get('values', [])

if not values:
    print 'No data found.'

scoodit_ingredients = pd.DataFrame(values[1:])
scoodit_ingredients = scoodit_ingredients.dropna(subset=[1])
for col in scoodit_ingredients.columns:
    scoodit_ingredients[col] = scoodit_ingredients[col].str.lower().str.strip()
print 'list of scoodit ingredients from google doc'
scoodit_ingredients.head()

,0,1,2,3,4,5,6
0,fruit,apple,None,None,None,None,None
1,fruit,apricot,None,None,None,None,None
2,fruit,banana,None,None,None,None,None
3,fruit,blackberry,None,None,None,None,None
4,fruit,blueberry,None,None,None,None,None


In [271]:
scoodit_wnids = {}

for ind, ing in scoodit_ingredients.iterrows():
    for cat in ing[1:].dropna():
        if cat in all_wnids['description'].values:
            wnids = list(all_wnids['wnid'].ix[all_wnids['description'] == cat])
            #print ind, cat, wnids,'\n'
            scoodit_wnids[ind] = wnids
            break

print 'total ingredients',len(scoodit_ingredients)
print 'wnids found',len(scoodit_wnids)
s_scoodit_wnids = pd.Series(scoodit_wnids) 

scoodit_ingredients_wnids = pd.merge(scoodit_ingredients,s_scoodit_wnids.to_frame(name='wnid'),left_index=True,right_index=True,how='outer')
print 'ingredients not in imagenet'
scoodit_ingredients_wnids.ix[scoodit_ingredients_wnids['wnid'].isnull()]

total ingredients 182
wnids found 175
ingredients not in imagenet


,0,1,2,3,4,5,6,wnid
16,fruit,galia melon,None,None,None,None,None,NaN
50,fruit,yuzu,None,None,None,None,None,NaN
99,vegetable,delicata squash,None,None,None,None,None,NaN
110,vegetable,kalamata olive,None,None,None,None,None,NaN
118,vegetable,nicoise olive,None,None,None,None,None,NaN
140,vegetable,stuffed olive,green pimento olives,None,None,None,None,NaN
258,meat,beef short rib,None,None,None,None,None,NaN


In [272]:
scoodit_ingredients_wnids.head()

,0,1,2,3,4,5,6,wnid
0,fruit,apple,None,None,None,None,None,"[n07739125, n12633994]"
1,fruit,apricot,None,None,None,None,None,"[n04971675, n07750872, n12640607]"
2,fruit,banana,None,None,None,None,None,"[n07753592, n12352287]"
3,fruit,blackberry,None,None,None,None,None,"[n07744811, n12653762]"
4,fruit,blueberry,None,None,None,None,None,"[n07743544, n12246232]"


In [275]:
l =  scoodit_ingredients_wnids['wnid'].dropna()
flatten = [item for sublist in l for item in sublist]

extra = {}
for ind,row in all_fr_veg_wnids.iterrows():
    if row['wnid'] not in flatten:
        extra[ind] = row

extra_fruits_df = pd.DataFrame.from_dict(extra).T.sort_values('wnid').drop_duplicates(subset='wnid')
print extra_fruits_df.shape
extra_fruits_df.head(5)

(511, 3)


,description,wnid,group
0,powdered mustard,n03994417,flavorers
2,relish,n07582441,flavorers
3,dip,n07582609,flavorers
4,bean dip,n07582811,flavorers
5,cheese dip,n07582892,flavorers


In [282]:
scoodit_veg_wnids ={}
wnids=[]
for ind, ing in scoodit_ingredients.iterrows():
    for cat in ing[1:].dropna():
        if cat in all_fr_veg_wnids['description'].values:
            wnids = list(all_fr_veg_wnids['wnid'].ix[all_fr_veg_wnids['description'] == cat])
            scoodit_veg_wnids[ind] = wnids
            wnids=[]
            break

print 'total ingredients',len(scoodit_ingredients)
print 'veg wnids found',len(scoodit_veg_wnids)
s_scoodit_veg_wnids = pd.Series(scoodit_veg_wnids) 

scoodit_ingredients_veg_wnids = pd.merge(scoodit_ingredients,s_scoodit_veg_wnids.to_frame(name='wnid'),left_index=True,right_index=True,how='outer')
print 'ingredients not in veg imagenet:',len(scoodit_ingredients_veg_wnids.ix[scoodit_ingredients_veg_wnids['wnid'].isnull()])
missing_in_veg =  scoodit_ingredients_veg_wnids.ix[scoodit_ingredients_veg_wnids['wnid'].isnull()]
missing_in_veg.head()

total ingredients 182
veg wnids found 124
ingredients not in veg imagenet: 58


,0,1,2,3,4,5,6,wnid
6,fruit,cape gooseberry,None,None,None,None,None,NaN
11,fruit,coconut,None,None,None,None,None,NaN
16,fruit,galia melon,None,None,None,None,None,NaN
50,fruit,yuzu,None,None,None,None,None,NaN
73,vegetable,arugula,None,None,None,None,None,NaN


In [295]:
missing_wnids ={}
wnids=[]
for ind, ing in missing_in_veg.iterrows():
    for cat in ing[1:].dropna():
        if cat in all_wnids['description'].values:
            wnids = list(all_wnids['wnid'].ix[all_wnids['description'] == cat])
            #print ind, cat, wnids,'\n'
            missing_wnids[ind] = wnids
            wnids=[]
            break

print 'missing in veg ingredients',len(missing_in_veg)
print 'found in all_wnids',len(missing_wnids)
s_missing_wnids = pd.Series(missing_wnids) 

scoodit_ingredients_wnids = scoodit_ingredients_veg_wnids
scoodit_ingredients_wnids['wnid'].loc[s_missing_wnids.index] = s_missing_wnids

print 'ingredients not in imagenet:',(len(scoodit_ingredients_wnids.
                                          ix[scoodit_ingredients_wnids['wnid']
                                             .isnull()]))
missing =  scoodit_ingredients_wnids.ix[scoodit_ingredients_wnids['wnid'].isnull()]

missing

missing in veg ingredients 58
found in all_wnids 51
ingredients not in imagenet: 7


,0,1,2,3,4,5,6,wnid
16,fruit,galia melon,None,None,None,None,None,NaN
50,fruit,yuzu,None,None,None,None,None,NaN
99,vegetable,delicata squash,None,None,None,None,None,NaN
110,vegetable,kalamata olive,None,None,None,None,None,NaN
118,vegetable,nicoise olive,None,None,None,None,None,NaN
140,vegetable,stuffed olive,green pimento olives,None,None,None,None,NaN
258,meat,beef short rib,None,None,None,None,None,NaN


In [296]:
scoodit_ingredients_wnids

,0,1,2,3,4,5,6,wnid
0,fruit,apple,None,None,None,None,None,[n07739125]
1,fruit,apricot,None,None,None,None,None,[n07750872]
2,fruit,banana,None,None,None,None,None,[n07753592]
3,fruit,blackberry,None,None,None,None,None,[n07744811]
4,fruit,blueberry,None,None,None,None,None,[n07743544]
5,fruit,cantaloupe,None,None,None,None,None,[n07755929]
6,fruit,cape gooseberry,None,None,None,None,None,[n12911079]
7,fruit,carambola,star fruit,None,None,None,None,[n07746551]
8,fruit,cherimoya,None,None,None,None,None,[n07761141]
9,fruit,cherry,None,None,None,None,None,[n07757132]


In [313]:

extra_ingredient_wnids = pd.merge(df,all_fr_veg)
extra_ingredient_wnids.tail(150)

,wnid,description,group
489,n07825312,chowchow,flavorers
490,n07825399,bread and butter pickle,flavorers
491,n07825496,pickle relish,flavorers
492,n07825597,piccalilli,flavorers
493,n07825717,sweet pickle,flavorers
494,n07825972,"soy sauce, soy",flavorers
495,n07826091,"Tabasco, Tabasco sauce",flavorers
496,n07826250,tomato paste,flavorers
497,n07826340,angelica,flavorers
498,n07826453,angelica,flavorers


In [314]:
len(extra_ingredient_wnids)

639

In [315]:
len(all_fr_veg)

639